In [3]:
import os
import subprocess
from glob import glob
from tqdm import tqdm


def generate_hstack_videos(input_videos, output_video):    
    ffmpeg_rep_files = [f' -i {f} ' for f in input_videos]
    hstack_args = f' -filter_complex hstack=inputs={len(input_videos)}' if len(input_videos) > 1 else ''
    ffmpeg_rep_cmd = f'ffmpeg -y -loglevel warning ' + ''.join(ffmpeg_rep_files) + f'{hstack_args} {output_video}'
    os.system(ffmpeg_rep_cmd)


def generate_vstack_videos(input_videos, output_video):
    ffmpeg_rep_files = [f' -i {f} ' for f in input_videos]
    vstack_args = f' -filter_complex vstack=inputs={len(input_videos)}' if len(input_videos) > 1 else ''
    ffmpeg_rep_cmd = f'ffmpeg -y -loglevel warning ' + ''.join(ffmpeg_rep_files) + f'{vstack_args} {output_video}'
    os.system(ffmpeg_rep_cmd)

def reorganize_motion_videos(num_pairs, num_rows, num_repetitions, input_dirs, output_dir):
    print("Reorganizing motion videos...")
    os.makedirs(output_dir, exist_ok=True)
    # Iterate over the pair indexes and repetitions
    for rep_idx in tqdm(range(num_repetitions), desc="Repetitions"):
        for pair_idx in tqdm(range(0, num_pairs * num_rows, num_rows), desc="Pairs"):
            output_videos = []
            # Build the list of input video paths for the current pair and repetitions
            for i in range(num_rows):
                input_videos = [f"{input_dir}/sample{pair_idx+i:02d}_rep{rep_idx:02d}.mp4" for input_dir in input_dirs]
                output_video = f"{output_dir}/sample{pair_idx+i:02d}_rep{rep_idx:02d}_grid.mp4"
                generate_hstack_videos(input_videos, output_video)
                output_videos.append(output_video)

            output_video = f"{output_dir}/pair{pair_idx:02d}_{pair_idx + num_rows - 1:02d}_rep{rep_idx:02d}_grid.mp4"        
            generate_vstack_videos(output_videos, output_video)        

    print("Done!")


In [9]:
num_pairs = 12
num_repetitions = 1
num_rows = 4
input_dirs = sorted(glob('/home/ctq566/motion-diffusion-model/save/p2p_quaternion_2_random/*'))
output_dir = "/home/ctq566/motion-diffusion-model/save/p2p_quaternion_2_random_grid/"

reorganize_motion_videos(num_pairs, num_rows, num_repetitions, input_dirs, output_dir)

Reorganizing motion videos...


Repetitions: 100%|██████████| 1/1 [01:08<00:00, 68.45s/it]

Done!


In [5]:
import numpy as np


data_1 = np.load('/home/ctq566/motion-diffusion-model/save/p2p_quaternion/001/sample01_rep00.npy')
data_2 = np.load('/home/ctq566/motion-diffusion-model/save/p2p_quaternion/007/sample01_rep00.npy')
(data_1 == data_2).all()

False

In [33]:
def check(sample_id):
    res = 0
    for i in range(1, 9):
        for j in range(i + 1, 10):
            data_1 = np.load(f'/home/ctq566/motion-diffusion-model/save/p2p_quaternion/{i:02d}/sample{sample_id:02d}_rep00.npy')
            data_2 = np.load(f'/home/ctq566/motion-diffusion-model/save/p2p_quaternion/{j:02d}/sample{sample_id:02d}_rep00.npy')
            res += (data_1 == data_2).all()
    return res


for i in range(1, 100, 4):
    print(check(0))


36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
